In [12]:
import pandas as pd
import numpy as np
from riotwatcher import LolWatcher, ApiError
from datetime import date
lol_watcher = LolWatcher('RGAPI-6e158fbd-afc2-49d3-bbcd-ceb263434efc')
queue = "RANKED_SOLO_5x5"
my_region = 'BR1'

In [9]:
try:
    master_queue = lol_watcher.league.masters_by_queue(my_region,queue)
    grandmaster_queue = lol_watcher.league.grandmaster_by_queue(my_region,queue)
    challenger_queue = lol_watcher.league.grandmaster_by_queue(my_region,queue)
except ApiError as err:
    if err.response.status_code == 429:
        print('We should retry in {} seconds.'.format(err.headers['Retry-After']))
        print('this retry-after is handled by default by the RiotWatcher library')
        print('future requests wait until the retry-after time passes')
    elif err.response.status_code == 404:
        print('Summoner with that ridiculous name not found.')
    else:
        raise

In [13]:
today = date.today().strftime("%d/%m/%Y")

In [29]:
master_queue_df = pd.DataFrame(master_queue['entries'])
master_queue_df['queue'] = 'master'
grandmaster_queue_df = pd.DataFrame(grandmaster_queue['entries'])
grandmaster_queue_df['queue'] = 'grandmaster'
challenger_queue_df = pd.DataFrame(challenger_queue['entries'])
challenger_queue_df['queue'] = 'challenger'

In [32]:
queues = pd.concat([grandmaster_queue_df,challenger_queue_df])

In [34]:
queues['date'] = today

In [35]:
queues.to_csv("queues_seed.csv",index=None)

In [30]:
print(f"Master User: {master_queue_df.summonerId.nunique()} GrandMaster Users : {grandmaster_queue_df.summonerId.nunique()} Challenger User: {challenger_queue_df.summonerId.nunique()}")

Master User: 2180 GrandMaster Users : 500 Challenger User: 500
